# TEST CODE

In [4]:
import requests
import xml.etree.ElementTree as ET
import xml.dom.minidom

url = "https://ecos.bok.or.kr/api/KeyStatisticList/sample/xml/kr/1/1/901Y009/"

response = requests.get(url)

if response.status_code == 200:
    try:
        contents = response.text
        ecosRoot = ET.fromstring(contents)
        
        if ecosRoot[0].text[:4] in ("INFO","ERRO"):
            print(ecosRoot[0].text + " : " + ecosRoot[1].text)
        else:
            dom = xml.dom.minidom.parseString(contents)
            pretty_xml_as_string = dom.toprettyxml(indent="   ")
            print(pretty_xml_as_string)
    except Exception as e:
        print(str(e))

<?xml version="1.0" ?>
<KeyStatisticList>
   
  
   <list_total_count number="true">100</list_total_count>
   
  
   <row_count number="true">1</row_count>
   
  
   <row array="true">
      
    
      <CLASS_NAME>가계</CLASS_NAME>
      
    
      <KEYSTAT_NAME>가구당월평균소득</KEYSTAT_NAME>
      
    
      <DATA_VALUE>6401718.413</DATA_VALUE>
      
    
      <CYCLE>2022Q4</CYCLE>
      
    
      <UNIT_NAME>원 </UNIT_NAME>
      
  
   </row>
   

</KeyStatisticList>



In [19]:
import requests
import json

url = "https://ecos.bok.or.kr/api/KeyStatisticList/sample/xml/kr/1/1/901Y009/"
service_name = "KeyStatisticList"
ecos_api_key = "sample"
resonse_formal = "json"
language = "kr"
start_no = "1"
end_no = "10"
table_code = "731Y003"
time_period = "D"
serach_from="20120101"
search_to="20230101"
item_code="0000003"


# API endpoint
url = "https://ecos.bok.or.kr/api/"+f"{service_name}/"+f"{ecos_api_key}/"+f"{resonse_formal}/"+f"{language}/"+\
    f"{start_no}/"+f"{end_no}/"+f"{table_code}/"+f"{time_period}/"+f"{serach_from}/"+f"{search_to}/"+f"{item_code}/"

# Make a GET request to the API
response = requests.get(url)

# Check the status code of the response
if response.status_code == 200:
    # If the request was successful, print the data
    clean_data = json.dumps(response.json(), indent = 4, ensure_ascii=False)
    print(clean_data)
else:
    # If the request was unsuccessful, print an error message
    print(f"Request failed with status code {response.status_code}")


{
    "KeyStatisticList": {
        "list_total_count": 100,
        "row_count": 10,
        "row": [
            {
                "CLASS_NAME": "환율",
                "KEYSTAT_NAME": "원/달러 환율(종가)",
                "DATA_VALUE": "1334.5",
                "CYCLE": "20230512",
                "UNIT_NAME": "원 "
            },
            {
                "CLASS_NAME": "환율",
                "KEYSTAT_NAME": "원/엔(100엔) 환율(매매기준율)",
                "DATA_VALUE": "981.85",
                "CYCLE": "20230512",
                "UNIT_NAME": "원 "
            },
            {
                "CLASS_NAME": "환율",
                "KEYSTAT_NAME": "원/유로 환율(매매기준율)",
                "DATA_VALUE": "1441.24",
                "CYCLE": "20230512",
                "UNIT_NAME": "원 "
            },
            {
                "CLASS_NAME": "환율",
                "KEYSTAT_NAME": "원/위안 환율(종가)",
                "DATA_VALUE": "191.77",
                "CYCLE": "20230512",
                "UNIT_NAME": "원 "
        

In [20]:
def get_product(KEY,STAT_CD,PERIOD,START_DATE,END_DATE):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    from lxml import html
    from urllib.request import Request, urlopen
    from urllib.parse import urlencode, quote_plus, unquote
    import pprint
    
    url = "http://ecos.bok.or.kr/api/StatisticSearch/{}/xml/kr/1/30000/{}/{}/{}/{}/"\
            .format(KEY,STAT_CD,PERIOD,START_DATE,END_DATE)
    
    response = requests.get(url).content.decode('utf-8')
    
    xml_obj = BeautifulSoup(response, 'lxml-xml')
    
    rows = xml_obj.findAll("row")
    return rows

In [51]:
data_dict={
    '521Y001':'뉴스심리지수',
    '731Y003':'원/달러',
    '200Y002':'주요지표/GDP',
    '802Y001':'주식시장',
    '901Y027':'경제활동인구',
    '403Y001':'수출금액지수', 
    '402Y014':'수출입물가지수',
    '404Y014':'생산자물가지수',
    '901Y009':'소비자물가지수',
    '901Y093':'부동산가격',
    '902Y003':'원자재' #금,아연,니켈,옥수수,알루미늄
}
KEY = 'TPWFYXDLNUXU9MNIA65E'

PERIOD = 'D'
START_DATE = '20200101'
END_DATE = '20230501'

In [67]:
item_list=[
    'STAT_CODE', #통계표 코드
    'ITEM_NAME', #통계명
    'ITEM_CODE1', #통계항목 1코드
    'ITEM_NAME1', #통계항목명 1
    'ITEM_CODE2',
    'ITEM_NAME2',
    'ITEM_CODE3',
    'ITEM_NAME3',
    'UNIT_NAME', #단위
    'TIME', #시점
    'DATA_VALUE' #값
]

result_list = list()
    
for k in data_dict.keys():
    rows = get_product(KEY,k,PERIOD,START_DATE,END_DATE)
    print(len(rows))
    
    for p in range(0,len(rows)):
        info_list = list()
        
        for i in item_list:
            try:
                individual_info = rows[p].find(i).text
            except:
                individual_info = ""
                
            info_list.append(individual_info)
        result_list.append(info_list)
    result_list
        

1217
7416
0
9064


In [52]:
import pandas as pd
from pandas import DataFrame
from datetime import datetime

result_df=DataFrame(result_list, columns=['통계표코드', #통계표 코드
    '통계명',
    '통계항목코드1',
    '통계항목명1',
    '통계항목코드2',
    '통계항목명2',
    '통계항목코드3',
    '통계항목명3',
    '단위', 
    '시점', 
    '값' ]).drop_duplicates()

# 값이 100 이상이면 긍정 이하면 부정
filtered_df = result_df[result_df['통계항목명1']=='뉴스심리지수']
filtered_df

,통계표코드,통계명,통계항목코드1,통계항목명1,통계항목코드2,통계항목명2,통계항목코드3,통계항목명3,단위,시점,값
0,521Y001,,A001,뉴스심리지수,,,,,,20200101,103.02
1,521Y001,,A001,뉴스심리지수,,,,,,20200102,99.42
2,521Y001,,A001,뉴스심리지수,,,,,,20200103,101.97
3,521Y001,,A001,뉴스심리지수,,,,,,20200104,102.9
4,521Y001,,A001,뉴스심리지수,,,,,,20200105,103.88
...,...,...,...,...,...,...,...,...,...,...,...
1212,521Y001,,A001,뉴스심리지수,,,,,,20230427,98.16
1213,521Y001,,A001,뉴스심리지수,,,,,,20230428,98.95
1214,521Y001,,A001,뉴스심리지수,,,,,,20230429,100.1
1215,521Y001,,A001,뉴스심리지수,,,,,,20230430,98.94


In [53]:
filtered_df = result_df[result_df['통계항목명1']=='원/달러(시가)']
filtered_df

,통계표코드,통계명,통계항목코드1,통계항목명1,통계항목코드2,통계항목명2,통계항목코드3,통계항목명3,단위,시점,값
1225,731Y003,,0000002,원/달러(시가),,,,,원,20200102,1153.8
1227,731Y003,,0000002,원/달러(시가),,,,,원,20200103,1158.5
1235,731Y003,,0000002,원/달러(시가),,,,,원,20200106,1168.1
1247,731Y003,,0000002,원/달러(시가),,,,,원,20200107,1168.5
1255,731Y003,,0000002,원/달러(시가),,,,,원,20200108,1170.3
...,...,...,...,...,...,...,...,...,...,...,...
8595,731Y003,,0000002,원/달러(시가),,,,,원,20230424,1332.5
8604,731Y003,,0000002,원/달러(시가),,,,,원,20230425,1331.5
8611,731Y003,,0000002,원/달러(시가),,,,,원,20230426,1339.1
8620,731Y003,,0000002,원/달러(시가),,,,,원,20230427,1338


In [54]:
filtered_df = result_df[result_df['통계항목명1']=='주요지표/GDP']
filtered_df

,통계표코드,통계명,통계항목코드1,통계항목명1,통계항목코드2,통계항목명2,통계항목코드3,통계항목명3,단위,시점,값


In [57]:
filtered_df = result_df[result_df['통계항목명1']=='KOSPI지수']
filtered_df


,통계표코드,통계명,통계항목코드1,통계항목명1,통계항목코드2,통계항목명2,통계항목코드3,통계항목명3,단위,시점,값
8635,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20200102,2175.17
8654,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20200103,2176.46
8663,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20200106,2155.07
8670,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20200107,2175.54
8677,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20200108,2151.31
...,...,...,...,...,...,...,...,...,...,...,...
17650,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20230424,2523.5
17663,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20230425,2489.02
17667,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20230426,2484.83
17683,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20230427,2495.81


In [58]:
filtered_df = result_df[result_df['통계항목명1']=='KOSDAQ지수']
filtered_df


,통계표코드,통계명,통계항목코드1,통계항목명1,통계항목코드2,통계항목명2,통계항목코드3,통계항목명3,단위,시점,값
8639,802Y001,,0089000,KOSDAQ지수,,,,,1996.07.01=1000,20200102,674.02
8650,802Y001,,0089000,KOSDAQ지수,,,,,1996.07.01=1000,20200103,669.93
8656,802Y001,,0089000,KOSDAQ지수,,,,,1996.07.01=1000,20200106,655.31
8666,802Y001,,0089000,KOSDAQ지수,,,,,1996.07.01=1000,20200107,663.44
8684,802Y001,,0089000,KOSDAQ지수,,,,,1996.07.01=1000,20200108,640.94
...,...,...,...,...,...,...,...,...,...,...,...
17646,802Y001,,0089000,KOSDAQ지수,,,,,1996.07.01=1000,20230424,855.23
17659,802Y001,,0089000,KOSDAQ지수,,,,,1996.07.01=1000,20230425,838.71
17672,802Y001,,0089000,KOSDAQ지수,,,,,1996.07.01=1000,20230426,830.44
17679,802Y001,,0089000,KOSDAQ지수,,,,,1996.07.01=1000,20230427,850.21


In [63]:
#KOSPI기준
filtered_df = result_df[result_df['통계항목명1']=='거래량(주식시장, 잠정치)']
filtered_df


,통계표코드,통계명,통계항목코드1,통계항목명1,통계항목코드2,통계항목명2,통계항목코드3,통계항목명3,단위,시점,값
8636,802Y001,,0002000,"거래량(주식시장, 잠정치)",,,,,만주,20200102,48861
8653,802Y001,,0002000,"거래량(주식시장, 잠정치)",,,,,만주,20200103,62253
8659,802Y001,,0002000,"거래량(주식시장, 잠정치)",,,,,만주,20200106,58747
8669,802Y001,,0002000,"거래량(주식시장, 잠정치)",,,,,만주,20200107,55933
8678,802Y001,,0002000,"거래량(주식시장, 잠정치)",,,,,만주,20200108,89962
...,...,...,...,...,...,...,...,...,...,...,...
17649,802Y001,,0002000,"거래량(주식시장, 잠정치)",,,,,만주,20230424,91450
17662,802Y001,,0002000,"거래량(주식시장, 잠정치)",,,,,만주,20230425,87577
17674,802Y001,,0002000,"거래량(주식시장, 잠정치)",,,,,만주,20230426,107565
17682,802Y001,,0002000,"거래량(주식시장, 잠정치)",,,,,만주,20230427,72274


In [65]:
#KOSPI 기준
filtered_df = result_df[result_df['통계항목명1']=='외국인 순매수(주식시장, 잠정치)']
filtered_df


,통계표코드,통계명,통계항목코드1,통계항목명1,통계항목코드2,통계항목명2,통계항목코드3,통계항목명3,단위,시점,값
8643,802Y001,,0030000,"외국인 순매수(주식시장, 잠정치)",,,,,억원,20200102,-64
8651,802Y001,,0030000,"외국인 순매수(주식시장, 잠정치)",,,,,억원,20200103,2618
8657,802Y001,,0030000,"외국인 순매수(주식시장, 잠정치)",,,,,억원,20200106,994
8667,802Y001,,0030000,"외국인 순매수(주식시장, 잠정치)",,,,,억원,20200107,1960
8683,802Y001,,0030000,"외국인 순매수(주식시장, 잠정치)",,,,,억원,20200108,2709
...,...,...,...,...,...,...,...,...,...,...,...
17652,802Y001,,0030000,"외국인 순매수(주식시장, 잠정치)",,,,,억원,20230424,-335
17660,802Y001,,0030000,"외국인 순매수(주식시장, 잠정치)",,,,,억원,20230425,86
17671,802Y001,,0030000,"외국인 순매수(주식시장, 잠정치)",,,,,억원,20230426,-925
17680,802Y001,,0030000,"외국인 순매수(주식시장, 잠정치)",,,,,억원,20230427,2801


In [66]:
filtered_df = result_df[result_df['통계항목명1']=='KOSPI지수']
filtered_df


,통계표코드,통계명,통계항목코드1,통계항목명1,통계항목코드2,통계항목명2,통계항목코드3,통계항목명3,단위,시점,값
8635,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20200102,2175.17
8654,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20200103,2176.46
8663,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20200106,2155.07
8670,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20200107,2175.54
8677,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20200108,2151.31
...,...,...,...,...,...,...,...,...,...,...,...
17650,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20230424,2523.5
17663,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20230425,2489.02
17667,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20230426,2484.83
17683,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20230427,2495.81


In [62]:
result_df

,통계표코드,통계명,통계항목코드1,통계항목명1,통계항목코드2,통계항목명2,통계항목코드3,통계항목명3,단위,시점,값
0,521Y001,,A001,뉴스심리지수,,,,,,20200101,103.02
1,521Y001,,A001,뉴스심리지수,,,,,,20200102,99.42
2,521Y001,,A001,뉴스심리지수,,,,,,20200103,101.97
3,521Y001,,A001,뉴스심리지수,,,,,,20200104,102.9
4,521Y001,,A001,뉴스심리지수,,,,,,20200105,103.88
...,...,...,...,...,...,...,...,...,...,...,...
17692,802Y001,,0003000,"거래대금(주식시장 , 잠정치)",,,,,억원,20230428,136393
17693,802Y001,,0002000,"거래량(주식시장, 잠정치)",,,,,만주,20230428,76256
17694,802Y001,,0001000,KOSPI지수,,,,,1980.01.04=100,20230428,2501.53
17695,802Y001,,0113000,"외국인 순매수(코스닥시장, 잠정치)",,,,,억원,20230428,-3649
